# Talk to your App

In majority of the examples, LLM Usage is shown as a chat-bot or personal assistant. 

A more powerful usage of LLM based Agents are when they interact with the system in the background and enrich it or take autonomous actions. This is conceptually similar to streaming analytics - with analytics being processed by Agents powered by LLMs.

You will see that for chat-bot type usage - if the AI infrastructure is down, things still work.
However for the Agents in backend - if the AI system is down, then the system will suffer some down time.

### Assumptions

* Pythons setup (I created a 3.11 venv)
* OpenAI Key (Granite struggled but perhaps not a fair comparions ollama/grantite q4 8b v `gpt-4o`

In [1]:
import openai
import re
import httpx
import os
import rich
import json
from openai import OpenAI
from agents import Agent, ModelSettings, function_tool,Runner
from rich.pretty import pprint


In [2]:
# Boilerplate for swapping in Granite via ollama
#model = "granite3-dense:8b"
#model = "granite3.1-dense:2b"
#client = OpenAI(
#     base_url='http://localhost:11434/v1',
#     api_key='ollama',
# ) 

from dotenv import load_dotenv
load_dotenv()

model = "gpt-4o"
client = OpenAI() 

In [3]:
# Quick test code - verify LLM conenctivity etc (disable via Raw)

chat_completion = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "Write a simple Python example class called User"}],
    temperature=0,
)
print(model)
print(f"{chat_completion.choices[0].message.content}")

gpt-4o
Certainly! Below is a simple example of a Python class called `User`. This class includes basic attributes like `username` and `email`, and a method to display user information.

```python
class User:
    def __init__(self, username, email):
        self.username = username
        self.email = email

    def display_user_info(self):
        print(f"Username: {self.username}")
        print(f"Email: {self.email}")

# Example usage:
if __name__ == "__main__":
    user1 = User("john_doe", "john@example.com")
    user1.display_user_info()
```

### Explanation:
- **`__init__` Method**: This is the constructor method that initializes a new instance of the `User` class with a `username` and `email`.
- **Attributes**: `username` and `email` are instance attributes that store the user's information.
- **`display_user_info` Method**: This method prints the user's information to the console.
- **Example Usage**: The `if __name__ == "__main__":` block is used to demonstrate how to create a

## Automated Error Handling

1.The process is started when an ansible job log completion data comes in. (as of now, we simulate this as a user input)
1. It examimes if there is any error. If no error, it ends
1. If there is an error:
    - Agent analyzes and recommends
    - Agent opens a jira ticket
    - Agent sends a slack message


In [4]:
from dataclasses import dataclass
from typing import Literal

from agents import Agent, ItemHelpers, Runner, TResponseInputItem, trace

model = "gpt-4o"

@dataclass
class Advisor:
    body: str

@dataclass
class Slacker:
    body: str

@dataclass
class JIRAer:
    body: str


advisor_agent = Agent(
    name="advisor_agent",
    instructions="You can look at the contents of an ansible log and spot the error. You will describe what the error is in a few crisp sentences so that a human can take corrective actions.",
    model = model,
    output_type=Advisor,
)

slack_agent = Agent(
    name="slack_agent",
    instructions="If there is an error captured in the input, you will always state - I have slacked the message. Also add the contents that in the provided input. Give a made up slack link. If there is no error, then just say - All is well, there is nothing to be done.",
    model = model,
    output_type=Slacker,
)

jira_agent = Agent(
    name="jira_agent",
    instructions="If there is an error captured in the input, you will always state - I have opened a JIRA ticket. Also add the contents that in the provided input. Give a made up JIRA Handle. If there is no error, then just say - All is well, there is nothing to be done.",
    model = model,
    output_type=JIRAer,
)



In [7]:
#msg = "all tasks successfully finished"
msg = "could not connect to the host as the password as expired. "
inputs = [{"content": msg, "role": "user"}]

#with trace("Router"):
#    story_outline_result = await Runner.run(advisor_agent,inputs)
#    #uncomment this to see the details
#    pprint(story_outline_result)
#    print("--------------------------")
#    print(story_outline_result.final_output)

with trace("Workflow"):
    print("----------Advisor Output----------")
    advisor_result = await Runner.run(advisor_agent,inputs)
    print(advisor_result.final_output.body)
    advisor_output: Advisor = advisor_result.final_output
    
    print("----------JIRA Output----------")
    jira_result = await Runner.run(jira_agent,advisor_output.body)
    print(jira_result.final_output.body)
    
    print("----------Slack Output----------")
    slack_result = await Runner.run(slack_agent,advisor_output.body)
    print(slack_result.final_output.body)

----------Advisor Output----------
The error indicates that Ansible is unable to connect to the host because the user's password has expired. To resolve this, update the password on the target host manually or via another management tool, ensuring compliance with security policies. Once updated, retry the connection with Ansible.
----------JIRA Output----------
I have opened a JIRA ticket. Details: The error indicates that Ansible is unable to connect to the host because the user's password has expired. To resolve this, update the password on the target host manually or via another management tool, ensuring compliance with security policies. Once updated, retry the connection with Ansible. 

JIRA Handle: ITOPS-1234
----------Slack Output----------
I have slacked the message: The error indicates that Ansible is unable to connect to the host because the user's password has expired. To resolve this, update the password on the target host manually or via another management tool, ensuring c

# Human-in-the-Loop
But LLMs hallucinate!

Yes, LLMs, just like traditional AI and human beings may not always give the right answer. To handle those kind of possible mistakes, we have processes in place.

In the above example, we are seamlessly blending human-in-the-loop when we open a JIRA ticket or Slack a message. Even if the contents of these are not entirely accurate, we do not lose much because a person can check and correct if needed. 

In distributed systems, there are lots of similar examples - which has been around us for a long time - to take care of possible errors: for example those which arise out of CAP Theorem related inconsistencies. 
